# 导入必要的库

In [1]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

* https://mxnet.incubator.apache.org/api/python/image/image.html
* http://zh.gluon.ai/chapter_computer-vision/fine-tuning.html
* http://zh.gluon.ai/chapter_computer-vision/kaggle-gluon-dog.html

# 定义预处理函数

In [2]:
ctx = mx.gpu()

preprocessing = [
    image.ForceResizeAug((224,224)),
    image.ColorNormalizeAug(mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225]))
]

def transform(data, label):
    data = data.astype('float32') / 255
    for pre in preprocessing:
        data = pre(data)
    
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

# 定义导出特征向量的函数

In [3]:
def get_features(net, data):
    features = []
    labels = []

    for X, y in tqdm(data):
        feature = net.features(X.as_in_context(ctx))
        features.append(feature.asnumpy())
        labels.append(y.asnumpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

* https://mxnet.incubator.apache.org/versions/master/api/python/gluon/model_zoo.html

# 导出训练集和测试集的特征向量

In [4]:
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('D:/dataset/dogbreed/train_valid_test/train/', transform=transform)
data = gluon.data.DataLoader(imgs, 16)

features_vgg, labels = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet, _ = get_features(models.densenet161(pretrained=True, ctx=ctx), data)

100%|██████████| 594/594 [07:36<00:00,  7.98s/it]


In [5]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('D:/dataset/dogbreed/train_valid_test/train/', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 16)

features_inception, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)

100%|██████████| 594/594 [03:36<00:00,  4.81s/it]


In [6]:
import h5py

with h5py.File('features.h5', 'w') as f:
    f['vgg'] = features_vgg
    f['resnet'] = features_resnet
    f['densenet'] = features_densenet
    f['inception'] = features_inception
    f['labels'] = labels

In [8]:
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('D:/dataset/dogbreed/train_valid_test/test/', transform=transform)
data = gluon.data.DataLoader(imgs, 16)

features_vgg, _ = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet, _ = get_features(models.densenet161(pretrained=True, ctx=ctx), data)

100%|██████████| 648/648 [08:09<00:00,  3.48s/it]


In [9]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('D:/dataset/dogbreed/train_valid_test/test/', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 16)

features_inception, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)

100%|██████████| 648/648 [03:30<00:00,  1.33s/it]


In [10]:
with h5py.File('features_test.h5', 'w') as f:
    f['vgg'] = features_vgg
    f['resnet'] = features_resnet
    f['densenet'] = features_densenet
    f['inception'] = features_inception